In [1]:
import numpy as np
import cv2
from PIL import Image
import open3d as o3d
import json

import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib import rcParams
from shapely.geometry import MultiPoint, box

# from pyquaternion.quaternion import Quaternion
from pyquaternion import Quaternion
import os.path as osp
from nuscenes import NuScenes

# Utils for Lidar and Radar
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.geometry_utils import view_points, BoxVisibility
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.scripts.export_2d_annotations_as_json import get_2d_boxes, post_process_coords
from utils import *

from typing import Tuple, List, Dict, Union

%matplotlib inline

In [2]:
#nusc = NuScenes(version='v1.0-mini', dataroot='/home/gus/Documents/AI_S/nuScenes/v1_0-mini', verbose=True)
nusc = NuScenes(version='v1.0-mini', dataroot='/home/sherlock/Documents/nuScenes/data', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.4 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [12]:
# Load the information from .txt file in JSON format
sample_path = '/home/sherlock/Documents/3d_obj_detection/anns/sample_0a0d6b8c2e884134a3b48df43d54c36a_file.txt'
# sample_path = '/home/gus/Documents/AI_S/ML/PF/sample_file/sample_ca9a282c9e77460f8360f564131a8af5_file.txt'

# This is a Dictionary of lists of dictionaries
data_json = load_file(sample_path)

74079cc158d24b789fb1b5ad428aa54d


In [16]:
for x in data_json['instance']:
    # Use dumps to print each 
    #print(json.dumps(x))
    
    #pc_anno_7e66fdf908d84237943c833e6c1b317a.txt is empty!!
    if x['pointcloud_path'] != "/home/gus/Documents/AI_S/ML/PF/anno_pc/pc_anno_7e66fdf908d84237943c833e6c1b317a.txt":
        # Load the Point Cloud from the .txt file
        pointcloud = load_pcl_txt(x['pointcloud_path'],x['pcl_shape'])

In [17]:
# Plotting the segmented point cloud
Velopoints = np.asarray(pointcloud.T, np.float32)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(Velopoints)

# visualizing the 3D point cloud
o3d.visualization.draw_geometries([pcd])